In [1]:
import pandas as pd

In [2]:
pd.__version__

'0.25.0'

In [2]:
import sys, sqlite3, pandas as pd
files = ['C:/Users/lianj/Desktop/Kaggle competition/Housing price/train.csv', 
         'C:/Users/lianj/Desktop/Kaggle competition/Housing price/train_modify.csv'] #these are the arguments we take
df1 = pd.read_csv(files[0])
df2 = pd.read_csv(files[1])
df3 = (df1 != df2).any(axis=None)
print('Differences in file:', df3)
df3 = (df1 != df2).any(1)
ne_stacked = (df1 != df2).stack()
changed = ne_stacked[ne_stacked]
changed.index.names = ['id', 'col']
print('Differences In:')
print(changed)

Differences in file: True
Differences In:
id    col        
0     Alley          True
      FireplaceQu    True
      PoolQC         True
      Fence          True
      MiscFeature    True
                     ... 
1459  Alley          True
      FireplaceQu    True
      PoolQC         True
      Fence          True
      MiscFeature    True
Length: 6966, dtype: bool


In [3]:
df3 = df1.eq(df2)
print(df3.all())
#print(df3.all(axis=1))
df4 = df3.all(axis=1)
df4 = pd.DataFrame(df4, columns=['Columns'])
print(df4[df4['Columns']==False])

Id                True
MSSubClass       False
MSZoning          True
LotFrontage      False
LotArea           True
                 ...  
MoSold            True
YrSold            True
SaleType          True
SaleCondition     True
SalePrice         True
Length: 81, dtype: bool
      Columns
0       False
1       False
2       False
3       False
4       False
...       ...
1455    False
1456    False
1457    False
1458    False
1459    False

[1460 rows x 1 columns]


In [4]:
#This is the correct mismtaching column


conn = sqlite3.connect(':memory:') #we are spinning an SQL db in memory
cur = conn.cursor()
chunksize = 10000
i=0
for file in files:
    i = i+1
    for chunk in pd.read_csv(file, chunksize=chunksize): #load the file in chunks in case its too big
        chunk.columns = chunk.columns.str.replace(' ', '_') #replacing spaces with underscores for column names
        chunk.to_sql(name='file' + str(i), con=conn, if_exists='append')
print('Comparing', files[0], 'to', files[1]) #Compare if all data from File[0] are present in File[1]
cur.execute( '''SELECT * FROM File1
                EXCEPT
                SELECT * FROM File2''')
i=0
for row in cur:
    print(row)
    i=i+1
if i==0: print('No Differences')
print('Comparing', files[1], 'to', files[0]) #Compare if all data from File[1] are present in File[0]
cur.execute( '''SELECT * FROM File2
                EXCEPT
                SELECT * FROM File1''')
i=0
for row in cur:
    print(row)
    i=i+1
if i==0: print('No Differences')
cur.close()

Comparing C:/Users/lianj/Desktop/Kaggle competition/Housing price/train.csv to C:/Users/lianj/Desktop/Kaggle competition/Housing price/train_modify.csv
(28, 29, 20, 'RL', 47.0, 16321, 'Pave', None, 'IR1', 'Lvl', 'AllPub', 'CulDSac', 'Gtl', 'NAmes', 'Norm', 'Norm', '1Fam', '1Story', 5, 6, 1957, 1997, 'Gable', 'CompShg', 'MetalSd', 'MetalSd', 'None', 0.0, 'TA', 'TA', 'CBlock', 'TA', 'TA', 'Gd', 'BLQ', 1277, 'Unf', 0, 207, 1484, 'GasA', 'TA', 'Y', 'SBrkr', 1600, 0, 0, 1600, 1, 0, 1, 0, 2, 1, 'TA', 6, 'Typ', 2, 'Gd', 'Attchd', 1957.0, 'RFn', 1, 319, 'TA', 'TA', 'Y', 288, 258, 0, 0, 0, 0, None, None, None, 0, 12, 2006, 'WD', 'Normal', 207500)
Comparing C:/Users/lianj/Desktop/Kaggle competition/Housing price/train_modify.csv to C:/Users/lianj/Desktop/Kaggle competition/Housing price/train.csv
(28, 29, 22, 'RL', 47.0, 16321, 'Pave', None, 'IR1', 'Lvl', 'AllPub', 'CulDSac', 'Gtl', 'NAmes', 'Norm', 'Norm', '1Fam', '1Story', 5, 6, 1957, 1997, 'Gable', 'CompShg', 'MetalSd', 'MetalSd', 'None', 0.0